In this notebook, I implement from scratch all operations required in a typical **Convolutional Neural Network**. By combining these functions in a correct way, we get a complete forward pass function of a CNN.

In [ ]:
import numpy as np
import h5py
import matplotlib.pyplot as plt

The following function implements the zero padding operation, which adds "pad" zeros to the border of all images of the dataset X. Note that the padding is applied to the height and width of an image while leaving the depth unaltered.

In [ ]:
def zero_pad(X, pad):
    X_pad = []
    for image in X:
        X_pad.append(np.pad(image, ((pad, pad), (pad, pad), (0, 0)), mode='constant', constant_values = (0, 0))) 
    X_pad = np.array(X_pad)
    
    return X_pad

The following function takes a slice of the volume and performs a convolution step on it with filter W and bias b.
The shape of the slide and the shape of the filter must match.

In [ ]:
def conv_single_step(a_slice_prev, W, b): #Apply one filter defined by parameters W on a single slice of the output activation and return a scalar 
    s = np.multiply(a_slice_prev, W).flatten().sum()
    Z = np.float64(s + b)
    return Z

Next, the conv_forward function performs a whole convolution step on an input volume and produces an output volume.

In [ ]:
def conv_forward(A_prev, W, b, hparameters):#Forward propagation for a convolution function

    (m, n_H_prev, n_W_prev, n_C_prev) = A_prev.shape
    (f, f, n_C_prev, n_C) = W.shape
    
    stride = hparameters["stride"]
    pad = hparameters["pad"]
    
    n_H = int((n_H_prev - f + 2*pad)/stride) + 1
    n_W = int((n_W_prev - f + 2*pad)/stride) + 1
    
    Z = np.zeros((m, n_H, n_W, n_C))
    
    A_prev_pad = zero_pad(A_prev, pad)
    for i in range(m):               # loop over the batch of training examples
        a_prev_pad = A_prev_pad[i]               # Select ith training example's padded activation
        for h in range(0, n_H):           # loop over vertical axis of the output volume
            # Find the vertical start and end of the current "slice" 
            vert_start = h*stride
            vert_end = vert_start+f
            for w in range(0, n_W):       # loop over horizontal axis of the output volume
                # Find the horizontal start and end of the current "slice" 
                horiz_start = w*stride
                horiz_end = horiz_start+f
                for c in range(0, n_C):   # loop over channels (= #filters) of the output volume
                    a_slice_prev = a_prev_pad[vert_start:vert_end,horiz_start:horiz_end,:]
                    weights = W[:,:,:,c]
                    biases = b[:,:,:,c]
                    Z[i, h, w, c] = conv_single_step(a_slice_prev, weights, biases)
                    

    cache = (A_prev, W, b, hparameters)
    return Z, cache

this last function implements the pooling operation on a given volume. Usually it serves to reduce the volume's width and width. It does not affect depth.

In [ ]:
def pool_forward(A_prev, hparameters, mode = "max"): #Forward pass of the pooling layer
    (m, n_H_prev, n_W_prev, n_C_prev) = A_prev.shape
    f = hparameters["f"]
    stride = hparameters["stride"]
    
    n_H = int(1 + (n_H_prev - f) / stride)
    n_W = int(1 + (n_W_prev - f) / stride)
    n_C = n_C_prev
    
    A = np.zeros((m, n_H, n_W, n_C))              
    
    for i in range(0, m):                         # loop over the training examples
        for h in range(0, n_H):                     # loop on the vertical axis of the output volume
            # Find the vertical start and end of the current "slice" (≈2 lines)
            vert_start = h*stride
            vert_end = vert_start+f
            
            for w in range(0, n_W):                 # loop on the horizontal axis of the output volume
                # Find the vertical start and end of the current "slice" (≈2 lines)
                horiz_start = w*stride
                horiz_end = horiz_start+f
                
                for c in range (0, n_C):            # loop over the channels of the output volume
                    a_prev_slice = A_prev[i, vert_start:vert_end, horiz_start:horiz_end, c]
                    if mode == "max":
                        A[i, h, w, c] = np.max(a_prev_slice)
                    elif mode == "average":
                        A[i, h, w, c] = np.mean(a_prev_slice)
    
    cache = (A_prev, hparameters)
    return A, cache

#### By stacking the above functions, we have implemented the whole CNN forward pass for a given input. 